# Taller Integrador - Modelo de Regresión Lineal Múltiple y Extensiones


## Carga de la Información

In [ ]:
### Cargar paquetes
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

### Importar información y depurar
from google.colab import drive
drive.mount('/content/drive')

## Carga de Informacion
ruta = "/content/drive/MyDrive/Analisis de Regresion/Bases de Datos/InformacionRetail.csv"
df = pd.read_csv(ruta, sep = ";")
df.info()

## Resumen Informacion
resumen_data = pd.DataFrame({
    "dtype":       df.dtypes.astype(str),
    "n_unique":    df.nunique(dropna=True),
    "n_null":      df.isna().sum(),
})

resumen_data["pct_null"] = (resumen_data["n_null"] / len(df) * 100).round(2)
resumen_data["unique_var"] = (resumen_data["n_unique"] <= 1)

# Variables con NA
vars_con_na = (resumen_data[resumen_data["pct_null"] > 0]
               .sort_values("pct_null", ascending=False)
               [["dtype","n_unique","n_null","pct_null"]])
print(vars_con_na)

# Variables Unique
vars_unique = (resumen_data[resumen_data["unique_var"]]
               .sort_values("n_unique")
               [["dtype","n_unique","n_null","pct_null"]])
print(vars_unique)

## Depurar Base de Datos
cols_quitar = ["CustomerID", "SignupDate", "LastPurchaseDate", "SeasonPref"]
df_mod = df.drop(columns=[c for c in cols_quitar if c in df.columns])

df_mod = df_mod.dropna()
df_mod.info()

## Modelo de Regresión Lineal Múltiple

In [ ]:
## Modelo de Regresión Lineal Múltiple
## Formula a Estimar
explicada = "NextMonthSpend"
explicativas = df_mod.columns.drop(explicada)
formula = explicada + " ~ " + " + ".join(explicativas)

## Generar Matrices Explicadas/Explicativas
from patsy import dmatrices
y, X = dmatrices(formula, data=df_mod, return_type="dataframe")

## Separacion de la Muestra
X_tr, X_te, y_tr, y_te = train_test_split(X, y,
                                          test_size=0.3,
                                          random_state=12345)

## Regresion Lineal Multiple
modelo_regresion   = sm.OLS(y_tr, X_tr).fit()
print(modelo_regresion.summary())

## Métricas de Bondad de Ajuste - TRAIN
r2_regresion_train = modelo_regresion.rsquared
ecm_regresion_train    = modelo_regresion.mse_resid
rmse_regresion_train   = np.sqrt(ecm_regresion_train)

## Métricas de Bondad de Ajuste - TEST
# Predicciones
y_hat_te = modelo_regresion.predict(X_te)
# R²
r2_regresion_test = r2_score(y_te.values, y_hat_te)

# ECM (MSE) y RMSE
ecm_regresion_test = mean_squared_error(y_te.values, y_hat_te)
rmse_regresion_test = np.sqrt(ecm_regresion_test)

# Resumen en tabla
resumen_regresion = pd.DataFrame({
    "Conjunto": ["TRAIN", "TEST"],
    "R2":       [r2_regresion_train,  r2_regresion_test],
    "ECM":      [ecm_regresion_train, ecm_regresion_test],
    "RMSE":     [rmse_regresion_train,rmse_regresion_test]
}).set_index("Conjunto")

print(resumen_regresion)

## Multicolinealidad

**Qué es.** Ocurre cuando algunos predictores están muy correlacionados entre sí. Consecuencias: coeficientes inestables y errores estándar elevados.

- **VIF (Variance Inflation Factor).** Para el predictor $X_j$:
$$
\mathrm{VIF}_j = \frac{1}{1 - R_j^2},
$$
donde $R_j^2$ es el R² de **regresar $X_j$ contra todos los demás predictores** (sin incluir la constante en la regresión auxiliar).
Reglas: $\mathrm{VIF} > 5$ (multicolinealidad moderada), $\mathrm{VIF} > 10$

- **Condition Index.** Si $X$ está **estandarizada por columnas** (media 0, desvío 1) y $s_i$ son sus **valores singulares**, entonces:
$$
\mathrm{CI}_i = \frac{s_{\max}}{s_i}.
$$
El **condition number global** es $\max_i \mathrm{CI}_i = s_{\max}/s_{\min}$.
Umbrales típicos: 10–15 (leve), 15–30 (moderado), **> 30** (alto).


**Interpretación rápida**
- VIF alto en una columna ⇒ colinealidad “local”.
- Condition number global alto ⇒ colinealidad “global”.
- Qué hacer: eliminar variables casi duplicadas, agrupar categorías raras y/o usar regularización (Ridge/Lasso).

## Ridge/ Lasso con Validación Cruzada

In [ ]:
## Validacion Cruzada
# Grilla de lambdas
alphas = np.logspace(-2, 2, 10)

# Ridge y Lasso con CV (sklearn)
ridge_cv = make_pipeline(StandardScaler(with_mean=False),
                         RidgeCV(alphas=alphas, cv=5))

lasso_cv = make_pipeline(StandardScaler(with_mean=False),
                         LassoCV(alphas=alphas, cv=5, max_iter=20000))

ridge_cv.fit(X_tr, y_tr.values.ravel())
lasso_cv.fit(X_tr, y_tr.values.ravel())

print("α* Ridge :", ridge_cv[-1].alpha_)
print("α* Lasso :", lasso_cv[-1].alpha_)

## Métricas
def metricas(name, model, Xtr, Xte, ytr, yte):
    yhat_tr = model.predict(Xtr)
    yhat_te = model.predict(Xte)
    r2_tr   = r2_score(ytr, yhat_tr)
    r2_te   = r2_score(yte, yhat_te)
    mse_tr  = mean_squared_error(ytr, yhat_tr)
    mse_te  = mean_squared_error(yte, yhat_te)
    rmse_tr = np.sqrt(mse_tr)
    rmse_te = np.sqrt(mse_te)
    return pd.Series({
        "R2_train":   r2_tr,  "R2_test":   r2_te,
        "ECM_train":  mse_tr, "ECM_test":  mse_te,
        "RMSE_train": rmse_tr,"RMSE_test": rmse_te
    }, name=name)

# Construir la tabla
tabla_metricas = pd.concat([
    metricas("MCO",   modelo_regresion,   X_tr, X_te, y_tr.values, y_te.values),
    metricas("Ridge", ridge_cv, X_tr, X_te, y_tr.values, y_te.values),
    metricas("Lasso", lasso_cv, X_tr, X_te, y_tr.values, y_te.values)
], axis=1).T.round(3)
print(tabla_metricas)

# Comparación Modelos

In [ ]:
## Modelos Ridge/ Lasso con alpha optimo
ridge = sm.OLS(y_tr, X_tr).fit_regularized(alpha=ridge_cv[-1].alpha_, L1_wt=0)
lasso = sm.OLS(y_tr, X_tr).fit_regularized(alpha=lasso_cv[-1].alpha_, L1_wt=1)

# Comparacion de Coeficientes
coef_ols   = pd.Series(modelo_regresion.params,   index=X_tr.columns, name="MCO")
coef_ridge = pd.Series(ridge.params, index=X_tr.columns, name="Ridge α*")
coef_lasso = pd.Series(lasso.params, index=X_tr.columns, name="Lasso α*")

coef_tab = pd.concat([coef_ols, coef_ridge, coef_lasso], axis=1).round(4)
print(coef_tab)

# Variables escogidas por Lasso
lasso_sel = coef_tab.loc[coef_tab["Lasso α*"].abs() > 0, "Lasso α*"].sort_values(key=np.abs, ascending=False)
print(f"Variables seleccionadas por Lasso (|β|>{0}): {lasso_sel.shape[0]}")
print(lasso_sel)

# Reponderacion Positiva por Ridge
ridge_sel = coef_tab.loc[coef_tab["Ridge α*"] > coef_tab["MCO"], ["MCO","Ridge α*"]]
print(ridge_sel)

# Predicciones

In [ ]:
### SUPONGAMOS QUE CONOCEMOS LAS VARIABLES EXPLICATIVAS UN MES PREVIO A LA PREDICCION
# Construir la matriz X para TODOS los clientes
from patsy import build_design_matrices, dmatrix
X_all = build_design_matrices([X.design_info], df_mod)[0]

# Predicciones con los tres modelos
pred_ols   = modelo_regresion.predict(X_all)
pred_ridge = ridge.predict(X_all)
pred_lasso = lasso.predict(X_all)

# Armar tabla cliente + predicciones
id_col = "CustomerID"
pred_df = pd.DataFrame(index=df_mod.index)
if id_col is not None:
    pred_df[id_col] = df.loc[df_mod.index, id_col]

pred_df["Pred_OLS"]   = np.asarray(pred_ols).ravel()
pred_df["Pred_Ridge"] = np.asarray(pred_ridge).ravel()
pred_df["Pred_Lasso"] = np.asarray(pred_lasso).ravel()

# Ensemble de Predicciones
pred_df["Pred_Promedio"] = pred_df[["Pred_OLS", "Pred_Ridge", "Pred_Lasso"]].mean(axis=1)

# Visualizar Tabla
print(pred_df.head(10))

# Descriptivos básicos de las predicciones promedio
desc = pred_df[["Pred_Promedio"]].describe().T
print(desc)